<a href="https://colab.research.google.com/github/JaDouth314/Big-data-challenge/blob/main/Part-one-code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://archive

In [1]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-29 22:51:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-11-29 22:51:51 (10.5 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [14]:
# Read the review dataset
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
review_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Read the next CSV file from an S3 bucket
url = 'https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_3/user_payment.csv'
spark.sparkContext.addFile(url)
user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True, inferSchema=True)

# Rename the id column to billing_id
user_payment_df = user_payment_df.withColumnRenamed("id", "billing_id")

# Show DataFrame
user_payment_df.show()

+----------+----------+--------------------+
|billing_id|  username|        cc_encrypted|
+----------+----------+--------------------+
|         1|   atuvey0|e1a4f985f7607bbae...|
|         2| bfrancke1|e1a4f985f7607bbae...|
|         3|   dduffy2|e1a4f985f7607bbae...|
|         4|   zsaker3|e1a4f985f7607bbae...|
|         5|   jargue4|e1a4f985f7607bbae...|
|         6|jcreeghan5|e1a4f985f7607bbae...|
|         7|  mrapley6|e1a4f985f7607bbae...|
|         8|    agant7|e1a4f985f7607bbae...|
|         9|  bgummow8|e1a4f985f7607bbae...|
|        10| nhaughin9|e1a4f985f7607bbae...|
|        11| sjurczika|e1a4f985f7607bbae...|
|        12|  knuschab|e1a4f985f7607bbae...|
|        13|   rantonc|e1a4f985f7607bbae...|
|        14|     jtodd|e1a4f985f7607bbae...|
|        15| dswinneye|e1a4f985f7607bbae...|
|        16|  ufernanf|e1a4f985f7607bbae...|
|        17|  ajoshamg|e1a4f985f7607bbae...|
|        18|cstiddardh|e1a4f985f7607bbae...|
|        19| mblizardi|e1a4f985f7607bbae...|
|        2

In [6]:
# Join the two DataFrames 
joined_df= user_data_df.join(user_payment_df, on="username", how="inner")
joined_df.show()

+----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|  username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   atuvey0|  1|      Andy|    Tuvey|      false| 12376 Darwin Circle|            New York|         1|e1a4f985f7607bbae...|
| bfrancke1|  2|   Bastian|  Francke|       true|14034 Summerview ...|          Washington|         2|e1a4f985f7607bbae...|
|   dduffy2|  3|    Dallis|    Duffy|      false| 8 Autumn Leaf Court|                Ohio|         3|e1a4f985f7607bbae...|
|   zsaker3|  4|      Zena|    Saker|       true|      3605 Gina Park|          California|         4|e1a4f985f7607bbae...|
|   jargue4|  5|      Jere|    Argue|       true|        8 Hauk Court|          New Jersey|         5|e1a4f985f7607bbae...|
|jcreegh

In [7]:
# Drop null values
dropna_df = joined_df.dropna(how='any')
dropna_df.show()

+----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|  username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   atuvey0|  1|      Andy|    Tuvey|      false| 12376 Darwin Circle|            New York|         1|e1a4f985f7607bbae...|
| bfrancke1|  2|   Bastian|  Francke|       true|14034 Summerview ...|          Washington|         2|e1a4f985f7607bbae...|
|   dduffy2|  3|    Dallis|    Duffy|      false| 8 Autumn Leaf Court|                Ohio|         3|e1a4f985f7607bbae...|
|   zsaker3|  4|      Zena|    Saker|       true|      3605 Gina Park|          California|         4|e1a4f985f7607bbae...|
|   jargue4|  5|      Jere|    Argue|       true|        8 Hauk Court|          New Jersey|         5|e1a4f985f7607bbae...|
|jcreegh

In [8]:
# Load a function that allows us to select columns
from pyspark.sql.functions import col

# Filter for only columns with active users
cleaned_df = dropna_df.filter(col("active_user") == True)
cleaned_df.show()

+------------+---+----------+----------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name| last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+----------+-----------+--------------------+--------------------+----------+--------------------+
|   bfrancke1|  2|   Bastian|   Francke|       true|14034 Summerview ...|          Washington|         2|e1a4f985f7607bbae...|
|     zsaker3|  4|      Zena|     Saker|       true|      3605 Gina Park|          California|         4|e1a4f985f7607bbae...|
|     jargue4|  5|      Jere|     Argue|       true|        8 Hauk Court|          New Jersey|         5|e1a4f985f7607bbae...|
|  jcreeghan5|  6|    Jethro|  Creeghan|       true|29047 Internation...|               Texas|         6|e1a4f985f7607bbae...|
|      agant7|  8| Anastasie|      Gant|       true|   4193 Dexter Drive|            Michigan|         8|e1a4f9

In [9]:
# Create a dataframe for the active_user table in our database
clean_user_df = cleaned_df.select(["billing_id", "first_name", "last_name", "username"])
clean_user_df.show()

+----------+----------+----------+------------+
|billing_id|first_name| last_name|    username|
+----------+----------+----------+------------+
|         2|   Bastian|   Francke|   bfrancke1|
|         4|      Zena|     Saker|     zsaker3|
|         5|      Jere|     Argue|     jargue4|
|         6|    Jethro|  Creeghan|  jcreeghan5|
|         8| Anastasie|      Gant|      agant7|
|        13|      Roth|     Anton|     rantonc|
|        14|     Jareb|       Tod|       jtodd|
|        18|    Carney|  Stiddard|  cstiddardh|
|        19|    Milzie|   Blizard|   mblizardi|
|        22|     Chuck|     Davio|     cdaviol|
|        25|    Maisie|      Pack|      mpacko|
|        27|   Trueman|   Spering|   tsperingq|
|        28|   Rosalyn|  Gascoyen|  rgascoyenr|
|        29|       Lev|     Basey|     lbaseys|
|        32|      Jean|   McPhail|   jmcphailv|
|        33|    Robbyn|   Macvain|   rmacvainw|
|        35|    Binnie|   de Grey|    bdegreyy|
|        36|   Phineas|Hindenburg|phinde

In [10]:
# Create a billing dataframe for the billing_info table in our database
clean_billing_df = cleaned_df.select(["billing_id", "street_address", "state", "username"])
clean_billing_df.show()

+----------+--------------------+--------------------+------------+
|billing_id|      street_address|               state|    username|
+----------+--------------------+--------------------+------------+
|         2|14034 Summerview ...|          Washington|   bfrancke1|
|         4|      3605 Gina Park|          California|     zsaker3|
|         5|        8 Hauk Court|          New Jersey|     jargue4|
|         6|29047 Internation...|               Texas|  jcreeghan5|
|         8|   4193 Dexter Drive|            Michigan|      agant7|
|        13|  143 Garrison Drive|District of Columbia|     rantonc|
|        14|      468 Vidon Lane|           Tennessee|       jtodd|
|        18|93776 Twin Pines ...|               Texas|  cstiddardh|
|        19|      4 Corscot Park|            Kentucky|   mblizardi|
|        22|       33 Elka Trail|              Nevada|     cdaviol|
|        25|0407 Northfield P...|           Minnesota|      mpacko|
|        27|98 Rockefeller Pa...|             Fl

In [11]:
# Create a payment dataframe for the payment_info table in our database
clean_payment_df = cleaned_df.select(["billing_id", "cc_encrypted"])
clean_payment_df.show()

+----------+--------------------+
|billing_id|        cc_encrypted|
+----------+--------------------+
|         2|e1a4f985f7607bbae...|
|         4|e1a4f985f7607bbae...|
|         5|e1a4f985f7607bbae...|
|         6|e1a4f985f7607bbae...|
|         8|e1a4f985f7607bbae...|
|        13|e1a4f985f7607bbae...|
|        14|e1a4f985f7607bbae...|
|        18|e1a4f985f7607bbae...|
|        19|e1a4f985f7607bbae...|
|        22|e1a4f985f7607bbae...|
|        25|e1a4f985f7607bbae...|
|        27|e1a4f985f7607bbae...|
|        28|e1a4f985f7607bbae...|
|        29|e1a4f985f7607bbae...|
|        32|e1a4f985f7607bbae...|
|        33|e1a4f985f7607bbae...|
|        35|e1a4f985f7607bbae...|
|        36|e1a4f985f7607bbae...|
|        37|e1a4f985f7607bbae...|
|        38|e1a4f985f7607bbae...|
+----------+--------------------+
only showing top 20 rows



Postgres Setup

In [12]:
# IMPORTANT: Replace each of these parameters with your own values for your AWS RDS instance
my_aws_endpoint = 'bigdatadatabase.ceb0jyms5b5v.us-east-1.rds.amazonaws.com' # This is my value; please replace with your own
my_aws_port_number = '5432' # Your value is likely the same, but please double check
my_aws_database_name = 'BigDataDataBase' # This is my value; please replace with your own
my_aws_username = 'bigdata' # Your value is likely the same, but please double check
my_aws_password = 'homework' # This is my value; please replace with your own


In [13]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to overwrite the existing data. Note that 'append' is probably a smarter choice
# in those situations where the PostgreSQL schema automatically generates the primary key. 
#  But 'overwrite' works best for this little demo. 
mode = 'overwrite' 


In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

clean_billing_df.write.jdbc(url=jdbc_url, table='billing_info', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

clean_payment_df.write.jdbc(url=jdbc_url, table='payment_info', mode=mode, properties=config)